In [1]:
#import libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
import pandas as pd
import datetime
import matplotlib.image as mpimg

Using TensorFlow backend.


In [55]:
#load data and name columns
data_csv = pd.read_csv("data/driving_log.csv",sep=",")
data_csv.columns=["center_image","right_image","left_image","steering_angle","throttle","break","speed"]

In [56]:
#create next frame time difference and session count based on time diff
#the session count can be used to associate training sessions and remove poor training sessions if need be
time_diff_list=[0]
training_session=[1]
k=1
for i in range(len(data_csv)-1):
    time_diff=(datetime.datetime.strptime(data_csv["center_image"].iloc[i+1][89:112],"%Y_%m_%d_%H_%M_%S_%f")-
         datetime.datetime.strptime(data_csv["center_image"].iloc[i][89:112],"%Y_%m_%d_%H_%M_%S_%f")).total_seconds()
    
    if  time_diff>0.5:
        k+=1
    time_diff_list.append(time_diff)
    training_session.append(k)
    

In [57]:
#add new columns to dataframe
data_csv["train_session"]=training_session
data_csv["time_diff"]=time_diff_list
data_csv=data_csv.loc[data_csv["train_session"]>=23]

In [58]:
#view the dataset
data_csv

,center_image,right_image,left_image,steering_angle,throttle,break,speed,train_session,time_diff
6478,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15021,23,1.562
6479,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15021,23,0.105
6480,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15021,23,0.115
6481,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15023,23,0.115
6482,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15021,23,0.116
6483,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15020,23,0.101
6484,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,0.000000,0.841635,0.0,30.15021,23,0.101
6485,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,-0.113858,0.841635,0.0,30.14808,23,0.116
6486,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,-0.084473,0.841635,0.0,30.14986,23,0.100
6487,/home/pepar/Python-Projects/Self_driving_car/C...,/home/pepar/Python-Projects/Self_driving_car/...,/home/pepar/Python-Projects/Self_driving_car/...,-0.071879,0.841635,0.0,30.15040,23,0.102


In [ ]:
# basic plot
print(min(data_csv["steering_angle"]))
print(max(data_csv["steering_angle"]))
plt.figure(figsize=(20,10))
plt.boxplot(data_csv["steering_angle"])

In [ ]:
#view steering anlges by training session
for unique_session in np.unique(data_csv["train_session"])[10:20]:
    plt.figure(figsize=(20,10))
    plt.plot(data_csv.loc[data_csv["train_session"]==unique_session]["steering_angle"])
    plt.show()

In [59]:
#create scaling image function
def scaling(img,scale_x,scale_y):
    """
    Input an image in a numpy array, scale of the x and ydirection as a decimal
    Outputs a numpy array as the scaled image
    """
    return cv2.resize(img,None,fx=scale_x, fy=scale_y, interpolation = cv2.INTER_LINEAR)

In [60]:
#create normalizing function to normailize augmented data set
def min_max_normalization(x,min,max):
    """
    This function takes an n by m array and normalizes each value based on the average of min and max values
    of the RBG scale (min=0 and max=255).
    
    It return an n by m array
    """
    avg_value=(max+min)/2.0
    norm_array = np.zeros(x.shape)+avg_value
    normalized_x= (x-norm_array)/norm_array
    return normalized_x

In [61]:
#split training set into training and validation set
from sklearn.cross_validation import train_test_split

y_train, y_validation = train_test_split(data_csv, test_size=0.2, random_state=88)

In [62]:
#used generator code from Behavioral cloning lesson
from sklearn.utils import shuffle
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for index, batch_sample in batch_samples.iterrows():
                name = batch_sample[0]
                center_image = min_max_normalization(scaling(mpimg.imread(name),0.5,0.5),0,255)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                images.append(np.fliplr(center_image))
                angles.append(-center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(y_train, batch_size=32)
validation_generator = generator(y_validation, batch_size=32)

In [47]:
model = Sequential()
# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
# Create the Sequential model
#convolutuonal layer with 10 layers and 3 by 3, 2x2 max pooling and activation
model.add(Convolution2D(10, 3, 3, border_mode='valid', input_shape=(80, 160, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Activation('relu'))

#convolutuonal layer with 20 layers and 3 by 3, 2x2 max pooling and activation
model.add(Convolution2D(20, 3, 3, border_mode='valid'))
model.add(MaxPooling2D((2,2)))
model.add(Activation('relu'))

#convolutuonal layer with 20 layers and 3 by 3, 2x2 max pooling and activation
model.add(Convolution2D(30, 3, 3, border_mode='valid'))
model.add(MaxPooling2D((2,2)))
model.add(Activation('relu'))

#a flatten layer
model.add(Flatten())

#1st layer dense to 800, activation and 50% dropout
model.add(Dense(800))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#2nd layer dense to 500, activation and 50% dropout
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#3rd layer dense to 200, activation and 50% dropout
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#4th layer dense to 100, activation and 50% dropout
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#5th layer dense to 64, activation and 50% dropout
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#6th layer dense to 32 and activation
model.add(Dense(32))
model.add(Activation('relu'))

#7th layer dense to 16 and activation
model.add(Dense(16))
model.add(Activation('relu'))

#8th layer dense to 1 for regression output
model.add(Dense(1))

In [63]:
#use adam optimizer and mean squared error as loss function and output mean absolute error as secondary measure
model.compile('adam', 'mean_squared_error', ['mae'])
#this line of code is taken from the behavioral learning lesson
model.fit_generator(train_generator, samples_per_epoch= len(y_train)*2, validation_data=validation_generator, nb_val_samples=len(y_validation)*2, nb_epoch=9)

Epoch 1/1
55206/55206 [==============================] - 707s - loss: 0.0210 - mean_absolute_error: 0.0818 - val_loss: 0.0221 - val_mean_absolute_error: 0.0848


In [65]:
#save model
model.save("model.h5")

In [46]:
#delete model for restart
del model
